In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import math
import os
import datetime
from pandas import DataFrame
import itertools
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
import torch
import xmltodict
import xml.etree.ElementTree as ET
import xml
from xml.etree.ElementTree import parse
from xml.parsers import expat
from xml.dom.minidom import parse, parseString
import psycopg2
import PyPDF2
import pikepdf
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
def createfiledirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

# **Define Functions to Import & Store Legislator-Committee Relationship Data from a Text File**

In [2]:
def findchair(com): 
    com=com.lstrip(' ')
    if 'vice' in com:
        com=com.split(', vi')[0]
        chair='vice chair'
    elif 'chair' in com or 'chiar' in com:
        com=com.split(', ch')[0]
        chair='chair'
    else:
        chair=''
    return com, chair# **Call Functions to Import & Store Legislator-Committee Relationship Data from a Text File**

In [3]:
def lastnamefix(lastname):
    if "'" in lastname:
        lastname=lastname.split("``")[0].rstrip(' ')
    return lastname

In [4]:
def longnames(name):
    if ',' in name:
        namelist=name.replace('.', '').split(',')
        name=namelist[0].replace('\n','').lstrip(' ')
        if 'of' not in namelist[1]:
            lastname=namelist[1].lstrip(' ')
            state=namelist[2].replace('\n','').split('of ')[1].lstrip(' ')
        else:
            lastname=namelist[1].replace('\n','').split(' of ')[0].lstrip(' ')
            state=namelist[1].replace('\n','').split(' of ')[1].lstrip(' ')
        lastname=lastnamefix(lastname)
    else:
        lastname=''
        state='' 
    return name, lastname, state

In [5]:
def loadrepdatafromxl(commlist):
    reppd=pd.DataFrame(columns=['Representative Name','Last Name','State', 'Committee', 'Role'])
    for nc in commlist.split('\n \n'):
        if len(nc.replace('.,','').split('...'))>1:
            name, lastname, state = longnames(list(filter(None, nc.replace('.,','').split('...')))[0].replace('\n','').lstrip(' '))
            replist=list(filter(None,list(filter(None,nc.replace('.,','').split('...')))[1].replace('  ', '').replace('\n', '').split('.')))
            for com in replist:
                com, chair=findchair(com)
                if 'Page' not in str(com):
                    reppd=reppd.append({'Representative Name': name,'Last Name':lastname,'State':state, 'Committee': com, 'Role':chair}, ignore_index=True)
        elif 'Page' not in str(nc.replace('.,','').split('...')):
            replist=list(filter(None,nc.replace('.,','').split('...')[0].replace('  ', '').replace('\n','').split('.')))
            if 'of' not in replist[0]:
                for com in replist:
                    com, chair=findchair(com)
                    reppd=reppd.append({'Representative Name': name,'Last Name':lastname,'State':state, 'Committee': com, 'Role':chair}, ignore_index=True)
            else:
                com, chair=findchair(replist[1].lstrip(' '))
                reppd=reppd.append({'Representative Name': replist[0].split(',')[0],'Last Name':lastnamefix(replist[0].split(',')[1].split(' of ')[0].lstrip(' ')),
                                    'State':replist[0].split(',')[1].split(' of ')[1],'Committee': com, 'Role':chair}, ignore_index=True)
    return reppd

# **Call Functions to Import & Store Legislator-Committee Relationship Data from a Text File**

In [7]:
username='joythompson'
rawdf="/Users/"+username+"/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/Raw_Data_Files/"
commlist= open(rawdf+ 'Committees.txt','r').read()
directoryfin=createfiledirectory('/Users/'+username+'/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/Bill_Rep_Parsing_to_SQL_Main/')

In [8]:
reppd= loadrepdatafromxl(commlist)
reppd.to_csv(directoryfin +'/unique_legislator_committee_relationship_by_line.csv')